# 第二课 历史记录处理

1. 怎么处理大模型的上下文
2. 有什么技巧去处理历史
3. 总结

In [1]:
# 初始化运行环境
require 'luo'

Luo.notebook_setup do |loader|
  loader.push_dir(File.join(__dir__, 'agents'))
end

puts Luo::VERSION

0.2.4


## 大模型的上下文如何处理

ChatGPT API 的设计使得处理历史记录相对简单。你可以将历史记录作为一个列表传递给 API 的 `messages` 参数。每条消息都是一个字典，包含两个键："role" 和 "content"。

- "role" 可以是 "system"、"user" 或 "assistant"，用于表示消息的角色。
- "content" 包含实际的文本内容。

下面是一个示例代码，展示如何将历史记录传递给 ChatGPT API：

```python
import openai

openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
```

在这个例子中，我们有一个系统角色的消息作为起始，然后是交替的用户和助手消息。你可以根据需要添加更多的消息。

注意，ChatGPT 是一种无状态模型，它只通过上下文理解消息，而不是维护对话状态。因此，当你向 API 发送消息时，确保包含完整的上下文历史，以便模型能够正确理解和回应消息。

### 怎么理解 无状态；不是维护对话状态呢

In [2]:
histroies = MemoryHistory.new(max_size: 12) #6轮对话

histroies.user("明天北京的天气怎么样")
histroies.assistant("北京明天的天气多云转晴，气温30度")

puts histroies.to_a # 尝试打印一下历史

{:role=>"user", :content=>"明天北京的天气怎么样"}
{:role=>"assistant", :content=>"北京明天的天气多云转晴，气温30度"}


In [3]:
# 我们并没有产生对话，你可以认为这些历史是我伪造的，那么如果我们继续问大模型我应该穿什么衣服会出现什么样的事情呢

messages = Messages.create(history: histroies).user(text: "那么我明天应该穿什么衣服呢？")

puts messages.to_a

{:role=>"user", :content=>"明天北京的天气怎么样"}
{:role=>"assistant", :content=>"北京明天的天气多云转晴，气温30度"}
{:role=>"user", :content=>"那么我明天应该穿什么衣服呢？"}


In [4]:
client = Xinghuo.new

response = client.chat(messages)
puts response

建议您穿轻便透气的衣服，如短袖T恤、短裤等。同时也要注意防晒和保护皮肤，可以携带太阳镜、遮阳帽等防晒用品。


## 用户喜好的秘密

1. 传统 NLP 用户喜好要不使用知识图谱，要不提取实体来做信息导入，处理难度大
2. 在大模型时代，用户喜好应该如何处理呢？

In [11]:
# 提供一种思路
user_preference = "点外卖的时候就帮我推荐粤菜"
histroies = MemoryHistory.new(max_size: 12) #6轮对话

histroies.user(user_preference)
histroies.assistant("好的记住了")

histroies.user("明天北京的天气怎么样")
histroies.assistant("明天北京下雨，气温30度")

messages = Messages.create(history: histroies).user(text: "我要叫外卖，你推荐一下吧")

client = Xinghuo.new

response = client.chat(messages)
puts response

# 无论你用 GPT 3.5 还是 星火效果都一般，除非你用GPT4，怎么改进

好的，请问您需要什么口味的菜品？


In [14]:
# 提供一种思路
user_preference = "点外卖的时候就帮我推荐粤菜"
histroies = MemoryHistory.new(max_size: 12) #6轮对话

histroies.user(user_preference)
histroies.assistant("好的记住了")

# histroies.user("明天北京的天气怎么样")
# histroies.assistant("明天北京下雨，气温30度")

messages = Messages.create(history: histroies).user(text: "我要叫外卖，你推荐一些菜吧")

client = Xinghuo.new

response = client.chat(messages)
puts response

# 无论你用 GPT 3.5 还是 星火效果都一般，除非你用GPT4，怎么改进

好的，以下是一些粤菜的推荐：

1. 叉烧饭：香甜可口的叉烧搭配白米饭，是一道经典的粤式美食。

2. 清蒸鲈鱼：新鲜的鲈鱼用姜丝、葱丝等调料清蒸，鲜嫩多汁，营养丰富。

3. 干炒牛河：选用宽粉和牛肉炒制而成，口感劲道，配上特制的辣椒酱更是美味。

4. 蚝油生菜：清淡爽口的生菜叶搭配浓郁的蚝油酱汁，非常适合夏天食用。

5. 糖醋排骨：外焦里嫩的排骨肉质鲜美，加上酸甜可口的糖醋汁，让人食欲大增。

希望以上推荐能够满足您的口味需求。


## 思考？

我并没有所谓的多轮对话的过程，就完成了上下文管理。

**tips**
1. 几乎所有大模型无论是私有部署还是公有云都是用这种形式去提供能力的
2. 模型其实并不存在对话历史这东西，有的只是上下文理解
3. 上下文是可以篡改的，已达到你要做的效果

## 作业

1. assistant这个东西是不是一定要和模型返回的结果是一致的呢？
2. 怎么优化历史？